In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.linear_model import Ridge, LinearRegression, Lasso
from sklearn.model_selection import cross_val_score
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#importing the data 
pb_investor_info = pd.read_csv('/content/drive/My Drive/pb_data/investor_info.csv', encoding= 'unicode_escape')
pb_investor_info.columns


pb_investor_info['last_investment_type'] = pd.to_numeric(pb_investor_info['last_investment_type'], errors='coerce')
pb_investor_info['last_investment_class'] = pd.to_numeric(pb_investor_info['last_investment_class'], errors='coerce')
pb_investor_info['primary_investor_type'] = pd.to_numeric(pb_investor_info['primary_investor_type'], errors='coerce')

pb_investor_info['investor_id'] = pb_investor_info['investor_id'].astype(float)
pb_investor_info['total_funds_open'] = pb_investor_info['total_funds_open'].astype(float)
pb_investor_info['total_funds_closed'] = pb_investor_info['total_funds_closed'].astype(float)
pb_investor_info['AUM'] = pb_investor_info['AUM'].astype(float)
pb_investor_info['dry_powder'] = pb_investor_info['dry_powder'].astype(float)
pb_investor_info['last_investment_valuation'] = pb_investor_info['last_investment_valuation'].astype(float)
pb_investor_info['last_investment_size'] = pb_investor_info['last_investment_size'].astype(float)

pb_investor_info['investments_last_5_yrs'] = pb_investor_info['investments_last_5_yrs'].astype(float)
pb_investor_info['investments_last_2_yrs'] = pb_investor_info['investments_last_2_yrs'].astype(float)
pb_investor_info['last_investment_class'] = pb_investor_info['last_investment_class'].astype(float)
pb_investor_info['last_investment_valuation'] = pb_investor_info['last_investment_valuation'].astype(float)
pb_investor_info['last_investment_size'] = pb_investor_info['last_investment_size'].astype(float)


pb_investor_id = pd.read_csv('/content/drive/My Drive/pb_data/investor_id.csv')
pb_investor_id.columns

pb_lp_info = pd.read_csv('/content/drive/My Drive/pb_data/lp_info.csv')
pb_lp_info.columns

pb_lp_id = pd.read_csv('/content/drive/My Drive/pb_data/lp_id.csv')
pb_lp_id.columns

pb_investor_lp = pd.read_csv('/content/drive/My Drive/pb_data/investor_lp.csv')
pb_investor_lp.columns

pb_investor_info.fillna(0,inplace=True)
pb_lp_info.fillna(0, inplace=True)

pb_lp_info.columns
pb_lp_info.dtypes

pb_lp_info['limited_partner_type'] = pd.to_numeric(pb_lp_info['limited_partner_type'], errors='coerce')
pb_lp_info['limited_partner_type'] = pb_lp_info['limited_partner_type'].astype(float)

pb_lp_info = pd.read_csv('/content/drive/My Drive/pb_data/lp_info.csv', encoding= 'unicode_escape')
pb_lp_info.columns
pb_lp_info.dtypes

pb_lp_info.fillna(0, inplace=True)


In [7]:
#for LP

# need to clean teh data in order to get this part to run


#split into train-validate set (90%) and test set (10%) with random_state=0

X1 = pb_lp_info[['ï»¿lp_id', 'cmmt_in_vc_funds', 'total_cmmt_in_vc_funds',
        'AUM', 'affliated_funds_num',
       'affliated_investors_num',  'total_cmmt']]
y1 = pb_lp_info.commitments

from sklearn.model_selection import train_test_split
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = 0.1, random_state = 0)

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import statsmodels.api as sm
linreg = LinearRegression()
scaler = StandardScaler()
minscaler = MinMaxScaler()
robust = RobustScaler()
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

pipe1 = make_pipeline(MinMaxScaler(),Lasso(max_iter=1000, tol=1e2))
param_grid = {'lasso__alpha': np.logspace(1,5,10)}
grid1 = GridSearchCV(pipe1, param_grid, cv=5, return_train_score=True)
grid1.fit(X1, y1)
features1=['lp_id', 'cmmt_in_vc_funds', 'total_cmmt_in_vc_funds',
        'AUM', 'affliated_funds_num',
       'affliated_investors_num',  'total_cmmt']

lasso1 = Lasso(alpha=grid1.best_params_['lasso__alpha'], tol=1e2)
lasso1.fit(X1_train, y1_train)
lasso1.score(X1_test, y1_test)

grid1.best_params_
grid1.best_score_

pd.Series(lasso1.coef_,index=features1).sort_values(ascending=False)


total_cmmt_in_vc_funds     1.767741e-02
total_cmmt                 1.599606e-04
AUM                        2.570849e-07
cmmt_in_vc_funds           0.000000e+00
affliated_funds_num        0.000000e+00
affliated_investors_num    0.000000e+00
lp_id                     -1.474137e-02
dtype: float64

this means that the most influential featuers in determining number of commitments are total commitments, total commitments in vc funds, and AUM - but very small coefficients... nothing really worth presenting on

In [ ]:
#for investors
#split into train-validate set (90%) and test set (10%) with random_state=0
X = pb_investor_info[['investor_id', 'investment_num', 'total_funds_open',
       'total_funds_closed', 'total_investments', 'dry_powder', 'last_investment_type',
       'last_investment_valuation', 'last_investment_size',
       'last_investment_class',
       'investments_last_2_yrs', 'investments_last_12_months',
       'investments_last_6_months', 'active_porfolio', 'exits',
       'last_closed_fund_size', 
       'primary_investor_type', 'max_fund_size', 'median_fund_size',
       'min_fund_size', 'fund_closed_num', 'total_investments_last_5_yrs',
       'total_investments_last_2_yrs', 'total_investments_last_12_months',
       'total_investments_last_6_months', 'total _active_portfolio',
       'total_exits', 'total_investments_last_5_yrs']]
y = pb_investor_info.AUM

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 0)

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import statsmodels.api as sm
linreg = LinearRegression()
scaler = StandardScaler()
minscaler = MinMaxScaler()
robust = RobustScaler()
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

pipe = make_pipeline(MinMaxScaler(),Lasso(max_iter=1000, tol=1e2))
param_grid = {'lasso__alpha': np.logspace(1,5,10)}
grid2 = GridSearchCV(pipe, param_grid, cv=5, return_train_score=True)
grid2.fit(X, y)


features=['investor_id', 'investment_num', 'total_funds_open',
       'total_funds_closed', 'total_investments', 'dry_powder', 'last_investment_type',
       'last_investment_valuation', 'last_investment_size',
       'last_investment_class',
       'investments_last_2_yrs', 'investments_last_12_months',
       'investments_last_6_months', 'active_porfolio', 'exits',
       'last_closed_fund_size', 
       'primary_investor_type', 'max_fund_size', 'median_fund_size',
       'min_fund_size', 'fund_closed_num', 'total_investments_last_5_yrs',
       'total_investments_last_2_yrs', 'total_investments_last_12_months',
       'total_investments_last_6_months', 'total _active_portfolio',
       'total_exits', 'total_investments_last_5_yrs']

lasso = Lasso(alpha=grid2.best_params_['lasso__alpha'], tol=1e2)
lasso.fit(X_train, y_train)
lasso.score(X_test, y_test)

grid2.best_params_
grid2.best_score_

pd.Series(lasso.coef_,index=features).sort_values(ascending=False)

total_funds_closed                  0.413064
exits                               0.080574
dry_powder                          0.074083
last_closed_fund_size               0.072561
total_exits                         0.033040
total_investments_last_5_yrs        0.028708
investment_num                      0.025190
investments_last_12_months          0.024515
total_investments_last_2_yrs        0.006251
last_investment_valuation           0.001579
last_investment_size                0.000868
median_fund_size                    0.000349
max_fund_size                       0.000189
min_fund_size                       0.000051
total_funds_open                    0.000000
total_investments_last_6_months     0.000000
total_investments_last_12_months    0.000000
total_investments_last_5_yrs        0.000000
fund_closed_num                     0.000000
last_investment_type                0.000000
primary_investor_type               0.000000
investments_last_6_months           0.000000
investment

the most important factors when predicting AUM are total funds closed, exits, dry powder, last closed fund size,total exits, and total invest,ents in the past five years